In [129]:
import pandas as pd
import numpy as np


input_dir = "options_data/atm_iv_earnings_master.csv"
master = pd.read_csv(input_dir)

In [130]:
#pre-processing 
master["earning_date"] = pd.to_datetime(master['earning_date'])
master = master.sort_values("earning_date")
#master = master[master["price_pre"] > 1]

In [131]:
master

,symbol,earning_date,option_type,expiry,strike,ticker,price_pre,price_post,price_change,iv_pre,iv_post,iv_crush,pre_file,post_file
151,GE,2022-01-25,call,2022-01-28,88.0,O:GE220128C00088000,7.65,2.22,-5.43,2.079645,0.849874,122.977156,2022-01-24,2022-01-26
234,RTX,2022-01-25,call,2022-01-28,87.0,O:RTX220128C00087000,2.54,3.30,0.76,0.692834,1.280458,-58.762392,2022-01-24,2022-01-26
235,RTX,2022-01-25,put,2022-01-28,87.0,O:RTX220128P00087000,1.58,0.50,-1.08,0.441542,0.199246,24.229637,2022-01-24,2022-01-26
152,GE,2022-01-25,put,2022-01-28,88.0,O:GE220128P00088000,0.70,0.94,0.24,0.197014,0.366396,-16.938166,2022-01-24,2022-01-26
204,LMT,2022-01-25,call,2022-01-28,360.0,O:LMT220128C00360000,16.80,31.20,14.40,1.111772,2.936297,-182.452421,2022-01-24,2022-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,UNH,2025-10-28,put,2025-10-31,350.0,O:UNH251031P00350000,4.90,2.35,-2.55,0.341854,0.232007,10.984670,2025-10-27,2025-10-29
403,UNH,2025-10-28,call,2025-10-31,350.0,O:UNH251031C00350000,21.12,8.30,-12.82,1.440070,0.798608,64.146194,2025-10-27,2025-10-29
169,UPS,2025-10-28,put,2025-10-31,75.0,O:UPS251031P00075000,0.10,0.01,-0.09,0.038136,0.008347,2.978935,2025-10-27,2025-10-29
157,UPS,2025-10-28,put,2025-10-31,75.0,O:UPS251031P00075000,0.10,0.01,-0.09,0.038136,0.008347,2.978935,2025-10-27,2025-10-29


In [132]:
#calcualtion s

#winrate of trade - number of negatives / total
#average win
#average loss


master["percentage_change"] = master["price_change"] / master["price_pre"]
master["win/loss"] = np.where(master["price_change"] < 0, 1, 0) #win/loss column

win_rate = master["win/loss"].mean()
print(f"Win Rate: {win_rate}")

# Average win and loss percentage
average_win = abs(master.loc[master["price_change"] < 0, "percentage_change"].mean())
average_loss = abs(master.loc[master["price_change"] > 0, "percentage_change"].mean())
print(f"Average Win: {average_win}")
print(f"Average Loss: {average_loss}")

#mean percentage change for wins
#mean percentage change for losses 
win = win_rate * average_win 
loss = ((1 - win_rate) * average_loss)

net_expected = win - loss
net_expected

Win Rate: 0.6791569086651054
Average Win: 0.7351565795637567
Average Loss: 1.3658883318339667


np.float64(0.06105083515746135)

In [133]:
#kelly criterion
#pricechange / pricepre <- wins and losses, and get the average for each 

def kelly_criterion(win_rate, avg_win, avg_loss):
    """
    Calculate Kelly Criterion for position sizing
    
    Parameters:
    win_rate (float): Probability of winning (0 to 1)
    avg_win (float): Average win percentage (e.g., 0.05 for 5%)
    avg_loss (float): Average loss percentage (e.g., -0.02 for -2%)
    
    Returns:
    float: Kelly percentage (0 to 1)
    """
    # Win/loss ratio (use absolute value since loss is negative)
    win_loss_ratio = abs(avg_win / avg_loss)
    
    # Kelly formula
    kelly = win_rate - (1 - win_rate) / win_loss_ratio
    
    # Cap at reasonable levels (0% to 100%)
    return max(0, min(kelly, 1))


kelly_pct = kelly_criterion(win_rate, average_win, average_loss)
print(f"Full Kelly: {kelly_pct:.1%}")
print(f"Half Kelly (safer): {kelly_pct/2:.1%}")

Full Kelly: 8.3%
Half Kelly (safer): 4.2%


In [140]:
#start with 100k trading account
#allocate half kelly bank roll per trade
#run over 2023 - 2025
#graph pnl over lifetime
#calculate the stev of account value 

#factor in brokerage costs

starting_amount = 100000
criterion_perc = kelly_pct / 2 #half kelly

backtest = master.copy().reset_index(drop=True)
backtest["percentage_won"] = -1*backtest["percentage_change"]
# Run backtest
#brokerage_cost = 0.01
account_value = starting_amount
account_values = [account_value]

for i in range(len(backtest)):
    position_size = account_value * criterion_perc

    contract_n = (position_size / backtest.loc[i, "price_pre"]).round() #integer
    #brokerage_cost = 0.02
    brokerage_cost = 0.01 * contract_n
    

    # Calculate trade P&L (brokerage costs for entry + exit)
    trade_pnl = position_size * backtest.loc[i, "percentage_won"] - (brokerage_cost * 2)
    
    # Update account value
    account_value += trade_pnl
    account_values.append(account_value)

# Store final account values (remove first value to align with trades)
backtest["MONEY"] = account_values[1:]

In [139]:
backtest


,symbol,earning_date,option_type,expiry,strike,ticker,price_pre,price_post,price_change,iv_pre,iv_post,iv_crush,pre_file,post_file,percentage_change,win/loss,percentage_won,MONEY
0,GE,2022-01-25,call,2022-01-28,88.0,O:GE220128C00088000,7.65,2.22,-5.43,2.079645,0.849874,122.977156,2022-01-24,2022-01-26,-0.709804,1,0.709804,102947.231603
1,RTX,2022-01-25,call,2022-01-28,87.0,O:RTX220128C00087000,2.54,3.30,0.76,0.692834,1.280458,-58.762392,2022-01-24,2022-01-26,0.299213,0,-0.299213,101668.174614
2,RTX,2022-01-25,put,2022-01-28,87.0,O:RTX220128P00087000,1.58,0.50,-1.08,0.441542,0.199246,24.229637,2022-01-24,2022-01-26,-0.683544,1,0.683544,104553.716877
3,GE,2022-01-25,put,2022-01-28,88.0,O:GE220128P00088000,0.70,0.94,0.24,0.197014,0.366396,-16.938166,2022-01-24,2022-01-26,0.342857,0,-0.342857,103065.226216
4,LMT,2022-01-25,call,2022-01-28,360.0,O:LMT220128C00360000,16.80,31.20,14.40,1.111772,2.936297,-182.452421,2022-01-24,2022-01-26,0.857143,0,-0.857143,99397.035796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,UNH,2025-10-28,put,2025-10-31,350.0,O:UNH251031P00350000,4.90,2.35,-2.55,0.341854,0.232007,10.984670,2025-10-27,2025-10-29,-0.520408,1,0.520408,184840.310429
423,UNH,2025-10-28,call,2025-10-31,350.0,O:UNH251031C00350000,21.12,8.30,-12.82,1.440070,0.798608,64.146194,2025-10-27,2025-10-29,-0.607008,1,0.607008,189499.054327
424,UPS,2025-10-28,put,2025-10-31,75.0,O:UPS251031P00075000,0.10,0.01,-0.09,0.038136,0.008347,2.978935,2025-10-27,2025-10-29,-0.900000,1,0.900000,196580.613176
425,UPS,2025-10-28,put,2025-10-31,75.0,O:UPS251031P00075000,0.10,0.01,-0.09,0.038136,0.008347,2.978935,2025-10-27,2025-10-29,-0.900000,1,0.900000,203926.810596
